## Image-Based 3D Modeling

based on [Image-Based 3D Modeling.ipynb](https://github.com/Mphasis-ML-Marketplace/Image-Based-3D-Modeling/blob/main/Image-Based%203D%20Modeling.ipynb).

This solution creates a 3D reconstruction from images using Structure-from-Motion (SfM) and Multi-View Stereo(MVS) techniques.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to For Seller to update: clean-sentiment-classification-labels. 

#### Contents:
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure dataset](#B.-Configure-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Execute optimization model](#3.-Execute-optimization-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Execute model](#3.2-Execute-model)
    1. [Visualize Output](#3.3-Inspect-the-Output-in-S3)
1. [Clean-up](#4.-Clean-up)
	1. [Unsubscribe to the listing (optional)](#Unsubscribe-to-the-listing-(optional))

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page **Image-Based 3D Modeling**
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
algo_arn = "colmap-listing-copy-04-20-1"

### 2. Prepare dataset

In [2]:
import os
import json 
import uuid
import boto3
import pickle
import base64
import tarfile
from pprint import pprint

import numpy as np
import pandas as pd

import urllib.request
from urllib.parse import urlparse

import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role

#### A. Dataset format expected by the algorithm

The algorithm requires data in the format as described:
* You need "input.zip" which contains parent_table_input.csv, child_table_input.csv, parent_table_variable_info.json, child_table_variable_info.json, input_parameters.json, relational_table_structure.json 
* parent_table_input.csv, child_table_input.csv are the original relational datasets
* parent_table_variable_info.json & child_table_variable_info.json should contain the variable type (ID, name, categorical, numerical) and variable sensitivity (True, False)
*  input_parameters.json should contain the parent_table_drop_cols, child_table_drop_cols and scale factor (no. of rows in synthetic compared to original)
*  relational_table_structure.json contains the relationship info between the tables
* For detailed instructions, please refer sample notebook and algorithm input details

#### B. Configure dataset

In [3]:
training_dataset="Input/train.zip"

#### C. Upload datasets to Amazon S3

In [4]:
role = get_execution_role()

In [5]:
sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

In [6]:
# training input location
common_prefix = "Image-Based 3D Modeling"
training_input_prefix = common_prefix + "/training-input-data"
TRAINING_WORKDIR = "Input" #Input directory in Jupyter Server
training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix) #uploads data from jupyter server to S3
print("Training input uploaded to " + training_input)

Training input uploaded to s3://sagemaker-us-east-2-786796469737/Image-Based 3D Modeling/training-input-data


## 3. Execute the training process

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to execute a training pipeline to get clean sentiment class labels using clean-sentiment-classification-labels Algorithm. 

### 3.1 Set up environment

In [7]:
output_location = 's3://{}/Image-Based 3D Modeling/{}'.format(bucket, 'Output')

### 3.2 Execute model

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [8]:
training_instance_type="ml.g4dn.xlarge"

In [9]:
#Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="ImageModeling",
    role=role,
    train_instance_count=1,
    train_instance_type=training_instance_type,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type=training_instance_type
)

#Run the training job.
estimator.fit({"training": training_input})

2023-05-05 08:57:30 Starting - Starting the training job...
2023-05-05 08:57:53 Starting - Preparing the instances for trainingProfilerReport-1683277050: InProgress
......
2023-05-05 08:58:53 Downloading - Downloading input data...
2023-05-05 08:59:13 Training - Downloading the training image.........
2023-05-05 09:00:54 Training - Training image download completed. Training in progress..==========
== CUDA ==
CUDA Version 11.7.0
Container image Copyright (c) 2016-2022, NVIDIA CORPORATION & AFFILIATES. All rights reserved.
This container image and its contents are governed by the NVIDIA Deep Learning Container License.
By pulling and using the container, you accept the terms and conditions of this license:
https://developer.nvidia.com/ngc/nvidia-deep-learning-container-license
A copy of this license is made available in this container at /NGC-DL-CONTAINER-LICENSE for your convenience.
*************************
** DEPRECATION NOTICE! **
*************************
THIS IMAGE IS DEPRECATED 

 Sweep 3: 0.6205s
 Sweep 4: 0.7799s
Iteration 2: 2.8259s
 Sweep 1: 0.6037s
 Sweep 2: 0.7859s
 Sweep 3: 0.6111s
 Sweep 4: 0.7596s
Iteration 3: 2.7604s
 Sweep 1: 0.5922s
 Sweep 2: 0.7601s
 Sweep 3: 0.5836s
 Sweep 4: 0.7295s
Iteration 4: 2.6655s
 Sweep 1: 0.5747s
 Sweep 2: 0.7331s
 Sweep 3: 0.5698s
 Sweep 4: 0.7133s
Iteration 5: 2.5910s
Total: 13.8288s
Writing photometric output for IMG-8915.jpg
Processing view 2 / 4 for IMG-8912.jpg
Reading inputs...
PatchMatch::Problem
-------------------
ref_image_idx: 0
src_image_idxs: 3 2 1
PatchMatchOptions
-----------------
max_image_size: 2400
gpu_index: 0
depth_min: 34.6476
depth_max: 86.9578
window_radius: 5
window_step: 1
sigma_spatial: 5
sigma_color: 0.2
num_samples: 15
ncc_sigma: 0.6
min_triangulation_angle: 1
incident_angle_sigma: 0.9
num_iterations: 5
geom_consistency: 0
geom_consistency_regularizer: 0.3
geom_consistency_max_cost: 3
filter: 0
filter_min_ncc: 0.1
filter_min_triangulation_angle: 3
filter_min_num_consistent: 2
filter_geom_cons

 Sweep 4: 1.5659s
Iteration 2: 5.4582s
 Sweep 1: 1.1481s
 Sweep 2: 1.5459s
 Sweep 3: 1.1297s
 Sweep 4: 1.5351s
Iteration 3: 5.3589s
 Sweep 1: 1.1376s
 Sweep 2: 1.5189s
 Sweep 3: 1.1143s
 Sweep 4: 1.5115s
Iteration 4: 5.2825s
 Sweep 1: 1.1283s
 Sweep 2: 1.4961s
 Sweep 3: 1.0991s
 Sweep 4: 2.1646s
Iteration 5: 5.8883s
Total: 27.5587s
Writing geometric output for IMG-8912.jpg
Processing view 3 / 4 for IMG-8913.jpg
Reading inputs...
PatchMatch::Problem
-------------------
ref_image_idx: 1
src_image_idxs: 2 3 0
PatchMatchOptions
-----------------
max_image_size: 2400
gpu_index: 0
depth_min: 35.9946
depth_max: 85.8546
window_radius: 5
window_step: 1
sigma_spatial: 5
sigma_color: 0.2
num_samples: 15
ncc_sigma: 0.6
min_triangulation_angle: 1
incident_angle_sigma: 0.9
num_iterations: 5
geom_consistency: 1
geom_consistency_regularizer: 0.3
geom_consistency_max_cost: 3
filter: 1
filter_min_ncc: 0.1
filter_min_triangulation_angle: 3
filter_min_num_consistent: 2
filter_geom_consistency_max_cost: 1


### 5. Perform Batch inference

In [ ]:
#upload the batch-transform job input files to S3
transform_input_folder = "Input/train.zip"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

In [ ]:
#Run the batch-transform job
transformer = estimator.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

In [10]:
#output is available on following path
transformer.output_path

's3://sagemaker-us-east-2-786796469737/Image-Based 3D Modeling/Output'

### 6. Clean-up

#### A. Delete the model

In [11]:
transformer.delete_model()

AttributeError: 'AlgorithmEstimator' object has no attribute 'delete_model'

In [ ]:
estimator.delete_model()

#### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.